In [1]:
import os,sys
import warnings
sys.path.insert(1, os.path.join(os.getcwd()  , '..'))
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
import shallowsim as sb
import pandas as pd
import math

In [3]:
args = sb.ModelArgs()
c = sb.Config()
gpu_blackwell_decode = sb.get_gpu_info('./device/gpu_info.csv', 
                                    decoding_mode=True, print_console=True) 

#gpu_blackwell_decode['H20'].pcie_bw = 25 #change bw to 1.6Tbps..

| gpu_type    |   sm |   comm_sm |   fp16 |   fp8 |   fp4 |   mem |   mem_bw |   nvlink_bw |   pcie_bw |   gpu_per_node |
|:------------|-----:|----------:|-------:|------:|------:|------:|---------:|------------:|----------:|---------------:|
| DGX-B300    |  160 |        20 |   3375 |  7500 | 15000 |   288 |     9600 |         900 |       100 |              8 |
| DGX-B200    |  160 |        20 |   2250 |  4500 |  9000 |   180 |     8000 |         900 |       100 |              8 |
| GB200-NVL72 |  160 |        20 |   2500 |  5000 | 10000 |   192 |     8000 |         900 |       100 |             72 |
| GB300-NVL72 |  160 |        20 |   3750 |  7500 | 15000 |   288 |     9600 |         900 |       100 |             72 |
| H200        |  132 |        20 |    989 |  1979 |     0 |   141 |     4800 |         450 |        50 |              8 |
| H800        |  132 |        20 |    989 |  1979 |     0 |    80 |     4000 |         200 |        50 |              8 |
| H20         |   78 |  

In [16]:
args.moe_inter_dim = 2048
args.n_activated_experts = 32
args.n_routed_experts = 1024

In [17]:
dfs = sb.decode_time_with_ep_list(args,gpu_blackwell_decode,c,print_console=False,fp8_combine=True)

# 推理性能天梯榜

In [18]:
dfs_o = dfs.groupby(['GPU','BatchSize'],as_index=False).apply(lambda t: t[t.Total==t.Total.max()]).sort_values(['Total'],ascending=False).reset_index(drop=True)
dfs_o.style.bar(subset=['TPS','Total'],color='#6495ED').applymap(sb.color_positive_red, subset=['Delta']).background_gradient(subset=['Comm_Impact'],cmap=sb.cm).format(precision=3) 

,GPU,TP,EP,BatchSize,DenseMLA,DenseMLP,SparseMLA,Combine,SharedExpert,RoutedExpert,Dispatch,COMP_SUM,COMM_SUM,Delta,TPOT,TPOT_O,TPS,TPS_O,Total,Total_O,Comm_Impact
0,GB300-NVL72,4,72,256,0.245,0.062,0.186,0.083,0.008,0.172,0.083,0.366,0.166,-0.028,22.136,22.136,45.175,45.175,11564.707,11564.707,0.000
1,GB300-NVL72,1,72,512,0.490,0.078,0.540,0.156,0.010,0.239,0.156,0.789,0.313,-0.237,47.472,47.472,21.065,21.065,10785.308,10785.308,0.000
2,GB300-NVL72,8,72,128,0.122,0.054,0.115,0.047,0.007,0.148,0.047,0.270,0.093,-0.029,16.201,16.201,61.726,61.726,7900.969,7900.969,0.000
3,GB200-NVL72,1,72,512,0.691,0.103,0.741,0.156,0.014,0.335,0.156,1.090,0.313,-0.442,65.584,65.584,15.248,15.248,7806.831,7806.831,0.000
4,GB200-NVL72,1,72,256,0.345,0.079,0.395,0.083,0.011,0.234,0.083,0.640,0.166,-0.240,38.400,38.400,26.042,26.042,6666.673,6666.673,0.000
5,GB200-NVL72,4,72,128,0.173,0.068,0.144,0.047,0.009,0.199,0.047,0.352,0.093,-0.060,21.131,21.131,47.324,47.324,6057.453,6057.453,0.000
6,GB300-NVL72,8,72,64,0.061,0.050,0.090,0.028,0.006,0.118,0.028,0.214,0.057,-0.040,12.745,12.745,78.461,78.461,5021.490,5021.490,0.000
7,GB200-NVL72,8,72,64,0.086,0.062,0.096,0.028,0.008,0.153,0.028,0.258,0.057,-0.048,15.388,15.388,64.988,64.988,4159.204,4159.204,0.000
8,DGX-B300,1,320,512,0.528,0.078,0.578,1.328,0.010,0.182,1.328,0.770,2.655,2.068,166.396,46.478,6.010,21.515,3076.989,11015.909,0.721
9,DGX-B300,4,320,256,0.264,0.062,0.191,0.669,0.008,0.101,0.669,0.300,1.338,1.139,84.445,18.375,11.842,54.422,3031.567,13931.936,0.782


# 查看单个CPU的性能

In [19]:
sb.df_filter(dfs,'H800',0).style.bar(subset=['TPS','Total'],color='#6495ED').applymap(sb.color_positive_red, subset=['Delta']).background_gradient(subset=['Comm_Impact'],cmap=sb.cm).format(precision=3) 

,GPU,TP,EP,BatchSize,DenseMLA,DenseMLP,SparseMLA,Combine,SharedExpert,RoutedExpert,Dispatch,COMP_SUM,COMM_SUM,Delta,TPOT,TPOT_O,TPS,TPS_O,Total,Total_O,Comm_Impact
126,H800,1,16,16,0.105,0.119,0.155,0.051,0.041,0.529,0.031,0.726,0.082,-0.115,42.776,42.776,23.378,23.378,374.042,374.042,0.000
127,H800,4,16,16,0.105,0.119,0.101,0.051,0.041,0.529,0.031,0.672,0.082,-0.061,39.628,39.628,25.234,25.234,403.750,403.750,0.000
128,H800,8,16,16,0.105,0.119,0.089,0.051,0.041,0.529,0.031,0.660,0.082,-0.049,38.946,38.946,25.676,25.676,410.820,410.820,0.000
129,H800,1,16,32,0.158,0.126,0.208,0.092,0.046,0.563,0.051,0.816,0.144,-0.110,48.197,48.197,20.748,20.748,663.942,663.942,0.000
130,H800,4,16,32,0.158,0.126,0.124,0.092,0.046,0.563,0.051,0.732,0.144,-0.026,43.315,43.315,23.087,23.087,738.773,738.773,0.000
131,H800,1,16,64,0.263,0.141,0.313,0.175,0.058,0.663,0.092,1.035,0.267,-0.104,61.226,61.226,16.333,16.333,1045.311,1045.311,0.000
217,H800,1,36,16,0.105,0.119,0.155,0.051,0.035,0.281,0.031,0.472,0.082,-0.109,28.032,28.032,35.673,35.673,570.768,570.768,0.000
218,H800,4,36,16,0.105,0.119,0.101,0.051,0.035,0.281,0.031,0.417,0.082,-0.054,24.885,24.885,40.185,40.185,642.958,642.958,0.000
219,H800,8,36,16,0.105,0.119,0.089,0.051,0.035,0.281,0.031,0.406,0.082,-0.043,24.203,24.203,41.317,41.317,661.076,661.076,0.000
220,H800,1,36,32,0.158,0.126,0.208,0.092,0.041,0.332,0.051,0.581,0.144,-0.106,34.547,34.547,28.946,28.946,926.279,926.279,0.000


In [24]:
gpu = 'H20'
tps_limit = 1

tdf = sb.df_filter(dfs,gpu,tps_limit=tps_limit)
sb.df_sort(tdf,value='Total',ascending=False).style.bar(subset=['TPS','Total'],color='#6495ED').applymap(sb.color_positive_red, subset=['Delta']).background_gradient(subset=['Comm_Impact'],cmap=sb.cm).format(precision=3) 

,GPU,TP,EP,BatchSize,DenseMLA,DenseMLP,SparseMLA,Combine,SharedExpert,RoutedExpert,Dispatch,COMP_SUM,COMM_SUM,Delta,TPOT,TPOT_O,TPS,TPS_O,Total,Total_O,Comm_Impact
0,H20,8,320,32,0.630,0.212,0.164,0.092,0.036,0.114,0.051,0.315,0.144,-0.057,20.771,20.771,48.144,48.144,1540.601,1540.601,0.000
1,H20,4,320,64,1.208,0.313,0.403,0.175,0.047,0.204,0.092,0.654,0.267,-0.183,42.475,42.475,23.543,23.543,1506.768,1506.768,0.000
2,H20,4,144,64,1.208,0.313,0.403,0.175,0.047,0.229,0.092,0.678,0.267,-0.183,43.908,43.908,22.775,22.775,1457.594,1457.594,0.000
3,H20,8,144,32,0.630,0.212,0.164,0.092,0.037,0.149,0.051,0.350,0.144,-0.058,22.854,22.854,43.756,43.756,1400.204,1400.204,0.000
4,H20,4,72,64,1.208,0.313,0.403,0.175,0.050,0.298,0.092,0.750,0.267,-0.185,48.073,48.073,20.802,20.802,1331.299,1331.299,0.000
5,H20,8,72,32,0.630,0.212,0.164,0.092,0.039,0.211,0.051,0.414,0.144,-0.059,26.532,26.532,37.691,37.691,1206.096,1206.096,0.000
6,H20,4,36,64,1.208,0.313,0.403,0.175,0.053,0.422,0.092,0.877,0.267,-0.188,55.429,55.429,18.041,18.041,1154.620,1154.620,0.000
7,H20,8,320,16,0.341,0.162,0.118,0.051,0.031,0.074,0.031,0.223,0.082,-0.067,14.464,14.464,69.137,69.137,1106.186,1106.186,0.000
8,H20,8,144,16,0.341,0.162,0.118,0.051,0.032,0.105,0.031,0.255,0.082,-0.068,16.303,16.303,61.338,61.338,981.406,981.406,0.000
9,H20,8,36,32,0.630,0.212,0.164,0.092,0.050,0.397,0.051,0.611,0.144,-0.070,37.949,37.949,26.351,26.351,843.238,843.238,0.000
